In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision import datasets
from glob import glob
import pandas as pd
import cv2
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim
from tqdm import tqdm
import ast 
from tqdm import tqdm
%matplotlib inline

class config:
    data_path = '../dataset/cats_and_dogs/data/'
    info_path = '../dataset/cats_and_dogs/pets_dataset_info.csv'
    device = torch.device("mps")
    batch_size = 16
    learning_rate = 0.01
    epochs = 10

In [2]:
train_transform = A.Compose([
    A.geometric.resize.LongestMaxSize(max_size=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2()
])

test_transform = A.Compose([  
    A.geometric.resize.LongestMaxSize(max_size=256),
    ToTensorV2()
])

In [1]:
class DataProcessor:
    def __init__(self, df_path=None):
        if df_path:
            self.df = pd.read_csv(df_path)
        self.cat_breeds = [
            'Abyssinian',
            'Bengal',
            'Birman',
            'Bombay',
            'British_Shorthair',
            'Egyptian_Mau',
            'Maine_Coon',
            'Persian',
            'Ragdoll',
            'Russian_Blue',
            'Siamese',
            'Sphynx'
        ]
        
        self.dog_breeds=[
            'american_bulldog',
            'american_pit_bull_terrier',
            'basset_hound',
            'beagle',
            'boxer',
            'chihuahua',
            'english_cocker_spaniel',
            'english_setter',
            'german_shorthaired',
            'great_pyrenees',
            'havanese',
            'japanese_chin',
            'keeshond',
            'leonberger',
            'miniature_pinscher',
            'newfoundland',
            'pomeranian',
            'pug',
            'saint_bernard',
            'samoyed',
            'scottish_terrier',
            'shiba_inu',
            'staffordshire_bull_terrier',
            'wheaten_terrier',
            'yorkshire_terrier'
        ]
        self.pet_label_to_id = {k:v for v, k in enumerate(['cat', 'dog'] + self.cat_breeds + self.dog_breeds)}
        self.pet_id_to_label = {v:k for k, v in self.pet_label_to_id.items()}
        

    def get_pet_and_breed_types(self, breeds):            
        labels = [0] * len(self.pet_label_to_id)
        for idx, breed in enumerate(breeds):
            pet_type = 0 if breed in self.cat_breeds else 1
            labels[pet_type] = 1        
            breed_type = self.pet_label_to_id[breed]        
            labels[breed_type] = 1        
        return labels

    def create_multi_label_df(self):        
        breeds = self.df['Breed'].apply(lambda x: ast.literal_eval(x))
        multi_label_cols = list(self.pet_label_to_id.keys())
        self.df[multi_label_cols] = breeds.apply(self.get_pet_and_breed_types).apply(pd.Series)
        df_mlbl = self.df.copy()
        return df_mlbl

pet_id_to_labels = DataProcessor().pet_id_to_label

In [4]:
class HarrisonPetDataset(Dataset):
    def __init__(self, imgs_path, info_df, train=True):
        self.imgs_path = imgs_path
        df = info_df
        
        sample_ids = info_df['Sample_ID']
        cat_id = list(df.columns.values).index('cat')

        self.image_paths, self.mask_paths, self.labels = [], [], []
        for sample_id in sample_ids:
            self.image_paths.append(f'{config.data_path}/{sample_id}/image.jpg')
            self.mask_paths.append(f'{config.data_path}/{sample_id}/mask.jpg')
            label = df[df['Sample_ID'] == sample_id].iloc[:, cat_id:].values[0]
            self.labels.append(label)

        # import pdb; pdb.set_trace()

        self.train=train
        
    def __len__(self):
        return len(self.image_paths)

    def read_image(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.train:
            image = train_transform(image=image)['image']
        else:
            image = test_transform(image=image)['image']
            

        image = image/ 255.        
        _, h, w = image.shape        
        square = torch.zeros((3, 256, 256))
        square[:, 0:h, 0:w] = image

        return square
        
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = self.read_image(image_path)
        label = self.labels[idx].astype(float)
        return image, label

In [5]:
df = DataProcessor(config.info_path).create_multi_label_df()
df = df.sample(frac=1)
n = int(df.shape[0] * 0.8)
train_df = df.iloc[:n, :].reset_index()
test_df = df.iloc[n:, :].reset_index()

train_ds = HarrisonPetDataset(
    imgs_path=config.data_path,
    info_df=train_df,
    train=True
)


test_ds = HarrisonPetDataset(
    imgs_path=config.data_path,
    info_df=test_df,
    train=False
)    

train_dataloader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=config.batch_size, shuffle=False)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class HarrisonNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)
        # for param in self.backbone.parameters():
        #     param.requires_grad = False
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 39)
        self.sigmode = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmode(x)
        return x

net = HarrisonNet().to(config.device)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=config.learning_rate)

In [7]:
def run_one_epoch(dl, train):
    losses = []
    for data in tqdm(dl, total=len(dl.dataset)//config.batch_size):
        inputs, labels = data
        labels = labels.to(torch.float32)
        
        inputs = inputs.to(config.device)
        labels = labels.to(config.device)
        optimizer.zero_grad()        
        
        if train:
            outputs = net(inputs)        
        else:
            with torch.no_grad():
                outputs = net(inputs)        
        
        loss = criterion(outputs.float(), labels.float())
        losses.append(loss.item())
        if train:
            loss.backward()
            optimizer.step()

    return np.mean(losses)

In [8]:
for epoch in range(config.epochs):
    net.train()
    train_loss = run_one_epoch(train_dataloader, train=True)
    net.eval()
    test_loss = run_one_epoch(test_dataloader, train=False)
    print (f'epoch {epoch} train loss {train_loss:.5f} test loss {test_loss: .5f}')

 28%|█████████████████████████████████▎                                                                                      | 126/454 [00:18<00:38,  8.63it/s]Corrupt JPEG data: premature end of data segment
455it [01:02,  7.31it/s]                                                                                                                                       
  6%|███████▌                                                                                                                  | 7/113 [00:00<00:08, 11.89it/s]Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
114it [00:09, 11.75it/s]                                                                                                                                       


epoch 0 train loss 0.21053 test loss  0.13329


  7%|████████▌                                                                                                                | 32/454 [00:04<00:48,  8.65it/s]Corrupt JPEG data: premature end of data segment
455it [00:53,  8.52it/s]                                                                                                                                       
  6%|███████▌                                                                                                                  | 7/113 [00:00<00:11,  9.37it/s]Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
114it [00:08, 12.85it/s]                                                                                                                                       


epoch 1 train loss 0.12296 test loss  0.10857


 41%|█████████████████████████████████████████████████▏                                                                      | 186/454 [00:21<00:31,  8.55it/s]Corrupt JPEG data: premature end of data segment
455it [00:53,  8.57it/s]                                                                                                                                       
  5%|██████▍                                                                                                                   | 6/113 [00:00<00:07, 14.42it/s]Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
114it [00:08, 14.13it/s]                                                                                                                                       


...

epoch 7 train loss 0.06518 test loss  0.05908


 26%|███████████████████████████████▍                                                                                        | 119/454 [00:13<00:38,  8.67it/s]Corrupt JPEG data: premature end of data segment
455it [00:52,  8.62it/s]                                                                                                                                       
  5%|██████▍                                                                                                                   | 6/113 [00:00<00:08, 13.03it/s]Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
114it [00:08, 13.74it/s]                                                                                                                                       


epoch 8 train loss 0.06152 test loss  0.05634


 15%|██████████████████▋                                                                                                      | 70/454 [00:08<00:44,  8.62it/s]Corrupt JPEG data: premature end of data segment
455it [00:52,  8.63it/s]                                                                                                                                       
  5%|██████▍                                                                                                                   | 6/113 [00:00<00:07, 14.13it/s]Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
114it [00:08, 14.02it/s]                                                                                                                                       

epoch 9 train loss 0.05872 test loss  0.05366


In [2]:
def run_eval(dl):    
    net.eval()    
    dl_labels, dl_outputs = [], []
    for data in tqdm(dl, total=len(dl.dataset)//config.batch_size):
        inputs, labels = data    
        
        labels = labels.data.cpu().numpy()
        inputs = inputs.to(config.device)
        
        with torch.no_grad():
            outputs = net(inputs)
        
        outputs = outputs.data.cpu().numpy()
        outputs = np.where(outputs > 0.5, 1.0, 0.0)    

        dl_labels.append(labels)
        dl_outputs.append(outputs)

    dl_labels = np.concatenate(dl_labels)
    dl_outputs = np.concatenate(dl_outputs)
                    
    metrics_dict = {}
    for i in range(39):
        tp = np.sum((dl_labels[:, i] == 1) & (dl_outputs[:, i] == 1))
        fp = np.sum((dl_labels[:, i] == 0) & (dl_outputs[:, i] == 1))    
        fn = np.sum((dl_labels[:, i] == 1) & (dl_outputs[:, i] == 0))        
        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        print (tp, fp, fn, precision, recall)
        metrics_dict[pet_id_to_labels[i]] = {
            'tp': tp,
            'fp': fp,
            'fn': fn,
            'precision': f'{precision: .2f}',
            'recall': f'{recall: .2f}'
        }

    return metrics_dict

metrics_dict = run_eval(test_dataloader)

NameError: name 'test_dataloader' is not defined

In [10]:
metrics_dict

{0: {'tp': 796, 'fp': 7, 'fn': 28, 'precision': ' 0.99', 'recall': ' 0.97'},
 1: {'tp': 1338, 'fp': 13, 'fn': 14, 'precision': ' 0.99', 'recall': ' 0.99'},
 2: {'tp': 14, 'fp': 2, 'fn': 52, 'precision': ' 0.88', 'recall': ' 0.21'},
 3: {'tp': 24, 'fp': 7, 'fn': 47, 'precision': ' 0.77', 'recall': ' 0.34'},
 4: {'tp': 29, 'fp': 19, 'fn': 30, 'precision': ' 0.60', 'recall': ' 0.49'},
 5: {'tp': 27, 'fp': 3, 'fn': 32, 'precision': ' 0.90', 'recall': ' 0.46'},
 6: {'tp': 32, 'fp': 6, 'fn': 43, 'precision': ' 0.84', 'recall': ' 0.43'},
 7: {'tp': 38, 'fp': 6, 'fn': 35, 'precision': ' 0.86', 'recall': ' 0.52'},
 8: {'tp': 9, 'fp': 3, 'fn': 42, 'precision': ' 0.75', 'recall': ' 0.18'},
 9: {'tp': 45, 'fp': 7, 'fn': 25, 'precision': ' 0.87', 'recall': ' 0.64'},
 10: {'tp': 13, 'fp': 6, 'fn': 69, 'precision': ' 0.68', 'recall': ' 0.16'},
 11: {'tp': 22, 'fp': 16, 'fn': 50, 'precision': ' 0.58', 'recall': ' 0.31'},
 12: {'tp': 46, 'fp': 5, 'fn': 21, 'precision': ' 0.90', 'recall': ' 0.69'},
 13:

In [24]:
def generate_labels(preds, id_to_label):
    labels = []
    # print (preds)
    for k, v in id_to_label.items():
        # print (k, v)
        if preds[k] == 1:
            labels.append(v)
    return labels

    # for i in range()

In [28]:

for _, row in test_df.iterrows():
    # print (row)
    img_id = row['Sample_ID']
    img_path = f'{config.data_path}/{img_id}/image.jpg'
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = test_transform(image=img)['image']
    img = img / 255
    _, h, w = img.shape        
    square = torch.zeros((3, 256, 256))
    square[:, 0:h, 0:w] = img
    square = square.unsqueeze(0)
    square = square.to(config.device)
    preds = net(square)[0]
    preds = preds.data.cpu().numpy()
    preds = np.where(preds > 0.5, 1.0, 0.0)
    preds = generate_labels(preds, pet_id_to_labels)
    print (row['Breed'], '|', preds)
    # break

['yorkshire_terrier'] | ['dog', 'yorkshire_terrier']
['chihuahua'] | ['dog', 'chihuahua']
['beagle'] | ['dog', 'beagle']
['miniature_pinscher'] | ['dog', 'miniature_pinscher']
['Birman'] | ['cat']
['yorkshire_terrier'] | ['dog', 'yorkshire_terrier']
['Russian_Blue'] | ['cat']
['japanese_chin'] | ['dog', 'japanese_chin']
['Abyssinian'] | ['cat']
['american_bulldog'] | ['dog']
['Maine_Coon'] | ['cat', 'Maine_Coon']
['British_Shorthair', 'german_shorthaired'] | ['cat', 'dog']
['Ragdoll'] | ['cat', 'Ragdoll']
['samoyed'] | ['dog', 'samoyed']
['samoyed'] | ['dog', 'samoyed']
['Persian'] | ['cat', 'Persian']
['yorkshire_terrier'] | ['dog', 'yorkshire_terrier']
['newfoundland'] | ['dog', 'newfoundland']
['Russian_Blue', 'english_cocker_spaniel'] | ['cat', 'dog']
['american_pit_bull_terrier'] | ['dog', 'american_pit_bull_terrier']
['Egyptian_Mau', 'samoyed'] | ['cat', 'dog', 'samoyed']
['British_Shorthair'] | ['cat', 'British_Shorthair']
['english_setter'] | ['dog']
['Maine_Coon'] | ['cat']
['

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


['great_pyrenees'] | ['dog']
['Egyptian_Mau'] | ['cat', 'Bengal', 'Egyptian_Mau']
['beagle'] | ['dog', 'beagle']
['Ragdoll', 'german_shorthaired'] | ['cat', 'dog']
['miniature_pinscher'] | ['dog', 'miniature_pinscher']
['leonberger'] | ['dog', 'leonberger']
['Persian'] | ['cat']
['Sphynx'] | ['cat', 'Sphynx']
['Bengal'] | ['cat']
['Abyssinian', 'japanese_chin'] | ['cat', 'dog']
['japanese_chin'] | ['dog', 'japanese_chin']
['saint_bernard'] | ['dog', 'saint_bernard']
['Birman'] | ['cat', 'Birman']
['Abyssinian', 'saint_bernard'] | ['cat', 'dog']
['beagle'] | ['dog']
['basset_hound'] | ['dog', 'basset_hound']
['pug'] | ['dog', 'pug']
['Russian_Blue'] | ['cat']
['Ragdoll'] | ['cat', 'Birman']
['pomeranian'] | ['dog', 'pomeranian']
['Maine_Coon'] | ['dog']
['keeshond'] | ['dog', 'keeshond']
['havanese'] | ['dog', 'havanese']
['american_bulldog'] | ['dog', 'american_bulldog']
['Egyptian_Mau'] | ['cat']
['Siamese'] | ['cat', 'Siamese']
['British_Shorthair', 'leonberger'] | ['cat', 'dog', 'le

In [26]:
row

index                                                         2893
Sample_ID                     5f11b88f-4d95-5899-83e6-29d7131ba6e5
Breed                                        ['yorkshire_terrier']
PET_ID                                                      [3052]
cat                                                              0
dog                                                              1
Abyssinian                                                       0
Bengal                                                           0
Birman                                                           0
Bombay                                                           0
British_Shorthair                                                0
Egyptian_Mau                                                     0
Maine_Coon                                                       0
Persian                                                          0
Ragdoll                                                       

In [78]:
F

<module 'torch.nn.functional' from '/Volumes/drive2/miniconda3/envs/amag-cv-utils/lib/python3.11/site-packages/torch/nn/functional.py'>

In [97]:
def run_inf(dl):    
    net.eval()    
    dl_labels, dl_outputs = [], []
    for data in tqdm(dl, total=len(dl.dataset)//config.batch_size):
        inputs, labels = data    
        
        labels = labels.data.cpu().numpy()
        inputs = inputs.to(config.device)
        
        with torch.no_grad():
            outputs = net(inputs)            
        
        outputs = outputs.data.cpu().numpy()
        print (outputs[0])
        outputs = np.where(outputs > 0.5, 1.0, 0.0)   
        print (outputs[0])
        

In [98]:
run_inf(test_dataloader)

  3%|███▏                                                                                                                      | 3/113 [00:00<00:09, 11.08it/s]

[0.36015    0.6718155  0.4567299  0.3745372  0.42137504 0.49607128
 0.52572364 0.529577   0.438038   0.6125372  0.5537614  0.462788
 0.4243333  0.6990985  0.48618773 0.3291325  0.51102227 0.742697
 0.7610983  0.61901784 0.4619072  0.7530376  0.5382932  0.7158735
 0.6491362  0.30097812 0.5521014  0.5468357  0.72904706 0.5107038
 0.402193   0.58574003 0.24355808 0.4752411  0.56965107 0.40296653
 0.73718    0.3695148  0.5623943 ]
[0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.]
[0.6306226  0.56091976 0.46419632 0.4657956  0.38424516 0.41081005
 0.4039913  0.54652935 0.5292125  0.4595863  0.5755789  0.28129506
 0.453626   0.59373873 0.43389854 0.4817635  0.44671997 0.40037355
 0.5484772  0.6251803  0.5093922  0.70347583 0.53481513 0.6465593
 0.52625424 0.3576294  0.49961543 0.64167607 0.4401964  0.51468855
 0.5364882  0.28867233 0.4413438  0.529009   0.58966285 0.31595197
 0.46820655 0.43654326 0.48768857]
[1. 1. 0. 0. 

  4%|█████▍                                                                                                                    | 5/113 [00:00<00:10, 10.58it/s]

[0.6134315  0.69467485 0.36633506 0.5010173  0.54560655 0.39722812
 0.5991449  0.61211896 0.5343737  0.53367084 0.46537894 0.3376828
 0.3683299  0.52937436 0.42384368 0.2576437  0.6395327  0.44289348
 0.7067353  0.63884795 0.3603011  0.6015632  0.545056   0.55868506
 0.4112286  0.4322033  0.5768818  0.70185643 0.5029157  0.46765456
 0.5312814  0.33480534 0.3899952  0.40237516 0.58870417 0.27486798
 0.52103734 0.48695087 0.668615  ]
[1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1.]
[0.46376246 0.76582456 0.5124251  0.40985036 0.5118194  0.42849508
 0.5516246  0.44493705 0.41230243 0.49992633 0.59938735 0.4190092
 0.48165026 0.5446135  0.37781507 0.26385543 0.6424505  0.6108627
 0.6825277  0.47244176 0.47242922 0.65570915 0.5415471  0.5627551
 0.59548736 0.4253814  0.6630862  0.5833134  0.66125727 0.49732366
 0.63897926 0.46836448 0.40476567 0.4063377  0.5578464  0.35821638
 0.6854916  0.44129655 0.66780835]
[0. 1. 1. 

  6%|███████▌                                                                                                                  | 7/113 [00:00<00:09, 11.11it/s]

...

[0.41046563 0.60291845 0.43305182 0.3674975  0.37596726 0.425203
 0.48425835 0.49252355 0.44031748 0.60151434 0.5532192  0.35040995
 0.505192   0.47429278 0.4951007  0.49102    0.493839   0.59596545
 0.6795262  0.54761404 0.4126486  0.6531718  0.5352343  0.49759784
 0.5026783  0.27055582 0.47707042 0.5161213  0.6442133  0.49466532
 0.5984885  0.4927913  0.43130386 0.37699875 0.5946816  0.3119494
 0.49876565 0.3841461  0.5424039 ]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
[0.63632387 0.70362836 0.5019542  0.5016437  0.5044076  0.29811472
 0.71149266 0.57908314 0.70080036 0.46171173 0.37486073 0.46295545
 0.48830357 0.4720773  0.4606666  0.31817374 0.5073731  0.66104144
 0.6565414  0.6598069  0.4092808  0.42628178 0.4774975  0.65481406
 0.4442283  0.54635674 0.62212485 0.681292   0.57064694 0.39591593
 0.51046723 0.4620484  0.41526878 0.36661536 0.62578607 0.34960312
 0.7438273  0.6735218  0.5369801 ]
[1. 1. 1.

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [00:08<00:00, 13.74it/s]

[0.53396463 0.68911356 0.405462   0.41673934 0.4882082  0.5460594
 0.54864234 0.60139686 0.45597428 0.6252409  0.5802785  0.4584566
 0.43285772 0.52582884 0.5795386  0.35314977 0.66196346 0.5659016
 0.7894531  0.45340186 0.33141282 0.6597571  0.6432269  0.6761514
 0.6281295  0.4378009  0.5350594  0.56573844 0.7048471  0.62568617
 0.58915436 0.4888137  0.43654224 0.5352352  0.43504435 0.28165683
 0.6614121  0.35250205 0.5966742 ]
[1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1.]
[0.48713997 0.7437786  0.45238727 0.5174126  0.5435746  0.41381097
 0.5671832  0.5577617  0.5035482  0.5883632  0.59600085 0.40534592
 0.40635276 0.59618986 0.543711   0.4272447  0.5546647  0.69454056
 0.7340308  0.5081002  0.3633721  0.6253256  0.5223092  0.6148694
 0.645226   0.36672634 0.57919717 0.5902189  0.6337363  0.44283143
 0.50356275 0.5139897  0.41815194 0.24263665 0.6293082  0.29087847
 0.5743304  0.54395205 0.6842483 ]
[0. 1. 0. 1

114it [00:08, 13.74it/s]                                                                                                                                       

[0.5634726  0.71363735 0.4007454  0.51548755 0.53337127 0.4567572
 0.4894047  0.5349171  0.5387966  0.518048   0.5327547  0.48155338
 0.52925134 0.5179092  0.5788035  0.4393776  0.50146437 0.46374777
 0.8181227  0.7319188  0.3901846  0.5955986  0.61247027 0.63509464
 0.583365   0.5428048  0.5963305  0.7271738  0.6003483  0.5551801
 0.47892156 0.49960655 0.34238586 0.47275588 0.4150524  0.34930503
 0.7217347  0.360317   0.5737571 ]
[1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]


In [10]:
dog_breeds=[
            'american_bulldog',
            'american_pit_bull_terrier',
            'basset_hound',
            'beagle',
            'boxer',
            'chihuahua',
            'english_cocker_spaniel',
            'english_setter',
            'german_shorthaired',
            'great_pyrenees',
            'havanese',
            'japanese_chin',
            'keeshond',
            'leonberger',
            'miniature_pinscher',
            'newfoundland',
            'pomeranian',
            'pug',
            'saint_bernard',
            'samoyed',
            'scottish_terrier',
            'shiba_inu',
            'staffordshire_bull_terrier',
            'wheaten_terrier',
            'yorkshire_terrier'
        ]

In [11]:
# self.pet_label_to_id = {k:v for v, k in enumerate(['cat', 'dog'] + self.cat_breeds + self.dog_breeds)}
str_to_id = {k: v for v, k in enumerate(dog_breeds)}
id_to_str = {v: k for k, v in str_to_id.items()}

In [14]:

list(str_to_id.keys())

['american_bulldog',
 'american_pit_bull_terrier',
 'basset_hound',
 'beagle',
 'boxer',
 'chihuahua',
 'english_cocker_spaniel',
 'english_setter',
 'german_shorthaired',
 'great_pyrenees',
 'havanese',
 'japanese_chin',
 'keeshond',
 'leonberger',
 'miniature_pinscher',
 'newfoundland',
 'pomeranian',
 'pug',
 'saint_bernard',
 'samoyed',
 'scottish_terrier',
 'shiba_inu',
 'staffordshire_bull_terrier',
 'wheaten_terrier',
 'yorkshire_terrier']

In [26]:
id_to_str

{0: 'american_bulldog',
 1: 'american_pit_bull_terrier',
 2: 'basset_hound',
 3: 'beagle',
 4: 'boxer',
 5: 'chihuahua',
 6: 'english_cocker_spaniel',
 7: 'english_setter',
 8: 'german_shorthaired',
 9: 'great_pyrenees',
 10: 'havanese',
 11: 'japanese_chin',
 12: 'keeshond',
 13: 'leonberger',
 14: 'miniature_pinscher',
 15: 'newfoundland',
 16: 'pomeranian',
 17: 'pug',
 18: 'saint_bernard',
 19: 'samoyed',
 20: 'scottish_terrier',
 21: 'shiba_inu',
 22: 'staffordshire_bull_terrier',
 23: 'wheaten_terrier',
 24: 'yorkshire_terrier'}